In [1]:
import altair as alt
import datetime as dt
import logging
import numpy as np
import pandas as pd
import requests

from requests.exceptions import RequestException
from json.decoder import JSONDecodeError
from itertools import chain

In [2]:
DOMAIN = "https://api-pub.bitfinex.com/"

# Decorator which jsonifies succesful requests and prints errors if otherwise
def bitfinex_api_handler(f):
    def inner(*args, **kwargs):
        try:
            response = f(*args, **kwargs)
            response.raise_for_status()  # Raises an HTTPError for bad requests
            return response.json()
        except JSONDecodeError:
            logging.error('JSON decoding error')
        except RequestException as e:
            logging.error(f'HTTP error: {e}')
        except Exception as e:
            logging.error(f'Unknown error: {e}')

        return []
    return inner

@bitfinex_api_handler
def fetch(start=None, end=None, ticker='BTCUSD'):
    url = f"{DOMAIN}/v2/trades/t{ticker}/hist"
    params = {'start': start, 'end': end, 'limit' : 10_000, 'sort': -1}
    return requests.get(url, params=params)

def timestamp_now_ms():
    now = dt.datetime.now()
    timestamp = dt.datetime.timestamp(now)
    return timestamp * 1000

def fetch_last(hours=2, ticker='BTCUSD'):
    timestamp = timestamp_now_ms()
    timestamp_stop = timestamp - hours * 3600 * 1000
    data = {}
    while timestamp > timestamp_stop:
        resp = fetch(end=timestamp, ticker=ticker)
        if resp:
            data[timestamp] = resp
            # data received is sorted new to old
            timestamp = resp[-1][1]
        else:
            break
    # flatten
    return list(chain.from_iterable(data.values()))

In [3]:
TICKER = 'BTCUSD'
LAST_HOURS = 4 * 24
data = fetch_last(hours=LAST_HOURS, ticker=TICKER)

ERROR:root:HTTP error: 429 Client Error: Too Many Requests for url: https://api-pub.bitfinex.com//v2/trades/tBTCUSD/hist?end=1702209458970&limit=10000&sort=-1


In [4]:
COLUMNS = {
    'ID': 'ID',
    'MTS': 'Timestamp',
    'AMOUNT': 'Q',
    'PRICE': 'P'
}

dfs = {}
# Data transform specific to Bitfinex
df = pd.DataFrame(data, columns=COLUMNS.values())
df['T'] = df['Timestamp'].map(lambda ts: dt.datetime.fromtimestamp(int(ts / 1000)))
df['Side'] = df['Q'].map(lambda q: 'B' if q > 0 else 'S')
df['Type'] = df['Side'].map(lambda x: 1 if x == 'S' else 0)
df['Q'] = df['Q'].map(abs)
df['V'] = df['Q'] * df['P']
df['V_buy'] = df['V'] * (df['Side'] == 'B')
df['V_sell'] = df['V'] - df['V_buy']
# Ascending order in time is important for OHLC
dfs['in'] = df.sort_values('T').reset_index(drop=True)
dfs['in']

,ID,Timestamp,Q,P,T,Side,Type,V,V_buy,V_sell
0,1471142443,1702209458970,0.276117,43754,2023-12-10 12:57:38,S,1,12081.211404,0.000000,12081.211404
1,1471142445,1702209515177,0.240472,43754,2023-12-10 12:58:35,S,1,10521.627202,0.000000,10521.627202
2,1471142447,1702209523371,0.318130,43754,2023-12-10 12:58:43,S,1,13919.450394,0.000000,13919.450394
3,1471142446,1702209523370,0.002000,43754,2023-12-10 12:58:43,S,1,87.508000,0.000000,87.508000
4,1471142464,1702209671679,0.002000,43754,2023-12-10 13:01:11,S,1,87.508000,0.000000,87.508000
...,...,...,...,...,...,...,...,...,...,...
149995,1476081902,1702850981701,0.001000,41896,2023-12-17 23:09:41,S,1,41.896000,0.000000,41.896000
149996,1476081903,1702850981704,0.013239,41895,2023-12-17 23:09:41,S,1,554.632823,0.000000,554.632823
149997,1476081904,1702850982284,0.028820,41893,2023-12-17 23:09:42,S,1,1207.356260,0.000000,1207.356260
149998,1476081909,1702851014896,0.050825,41892,2023-12-17 23:10:14,B,0,2129.142886,2129.142886,0.000000


In [5]:
def copy(f):
    def inner(df, *args, **kwargs):
        _df = df.copy()
        return f(_df, *args, **kwargs)
    return inner

def upsample(df, resolution=10_000):
    def discretize(df, resolution=resolution):
        # 136K -> 13 * [10K] + [6K]
        split_volume = lambda v: int(v // 1000) * [1000] + [v % 1000]
        return df['V'].map(split_volume)

    def recalculate(df):
        df['Q'] = df['V'] / df['P']
        df['V_buy'] = df['V'] * (1 - df['Type'])
        df['V_sell'] = df['V'] * df['Type']
        df['Q_buy'] = df['Q'] * (1 - df['Type'])
        df['Q_sell'] = df['Q'] * df['Type']
        return df

    return (df
            .assign(V_split = discretize)
            .explode('V_split')
            .drop('V', axis=1)
            .rename(columns={'V_split': 'V'})
            .pipe(recalculate)
            )

def bin_by_cumulative_volume(df, bin_size):
    max_cum_vol = df['V'].cumsum().iloc[-1]
    stop = np.ceil(max_cum_vol) + bin_size
    bins = np.arange(0, stop, bin_size)
    binned = pd.cut(df['V'].cumsum(), bins=bins)
    df['V_upper_bound'] = binned.map(lambda bin: bin.right)
    return df

def flatten_columns(df):
    df.columns = ['_'.join(col).strip() for col in df.columns.values]
    return df

def aggregate_ohlc(df):
    return (df.groupby('V_upper_bound')
              .agg({
                'P': ['first', 'max', 'min', 'last'],
                'Q': 'sum',
                'V': 'sum',
                'V_buy': 'sum',
                'V_sell': 'sum',
                'T': 'last'
              }).pipe(flatten_columns)
               .rename(columns={
                  'P_first': 'open',
                  'P_max': 'high',
                  'P_min': 'low',
                  'P_last': 'close',
                  'Q_sum': 'Q',
                  'V_sum': 'V',
                  'V_buy_sum': 'V_buy',
                  'V_sell_sum': 'V_sell',
                  'T_last': 'T'
              })
          )

def clean(df):
    return df.fillna(method='ffill').reset_index(drop=True)



BIN_SIZE = 10_000_000
RESOLUTION = BIN_SIZE // 100

dfs['v'] = (dfs['in']
            .pipe(upsample, resolution=RESOLUTION)
            .pipe(bin_by_cumulative_volume, bin_size=BIN_SIZE)
            .pipe(aggregate_ohlc)
            .pipe(clean)
            .assign(D=lambda df: df['V_buy'] - df['V_sell'])
            )

dfs['v'].head()

,open,high,low,close,Q,V,V_buy,V_sell,T,D
0,43754,44066,43744,43865,227.921629,9.999114e+06,2.589352e+06,7.409762e+06,2023-12-10 23:05:23,-4.820410e+06
1,43865,43951,43851,43950,227.657164,1.000016e+07,1.814536e+06,8.185620e+06,2023-12-10 23:08:11,-6.371084e+06
2,43950,43958,43623,43767,227.861828,9.999937e+06,1.938972e+06,8.060964e+06,2023-12-11 01:29:06,-6.121992e+06
3,43767,43793,43329,43333,229.468551,1.000077e+07,6.341028e+06,3.659744e+06,2023-12-11 02:08:34,2.681283e+06
4,43333,43445,42718,42760,231.445347,9.999226e+06,3.068759e+06,6.930466e+06,2023-12-11 03:07:07,-3.861707e+06


In [6]:
def plot_candles(df, x_axis=None, width=800, height=400):
    green = 'forestgreen'
    red = 'rgb(255, 72, 51)'
    darkred = 'firebrick'
    white = 'white'

    open_close_fill = alt.condition(
        "datum.open <= datum.close",
        alt.value(white),
        alt.value(red)
    )

    open_close_rule = alt.condition(
        "datum.open <= datum.close",
        alt.value(green),
        alt.value(darkred)
    )

    open_close_stroke = alt.condition(
        "datum.open <= datum.close",
        alt.value(green),
        alt.value(darkred)
    )

    base = alt.Chart(df, width=width, height=height).encode(
        x=x_axis,
        color=open_close_fill,
    )

    rule = base.mark_rule().encode(
        alt.Y(
            'low:Q',
            title='Price',
            scale=alt.Scale(zero=False),
        ),
        alt.Y2('high:Q'),
        color=open_close_rule
        )

    bar = base.mark_bar(
        size=(width / len(df)) * 0.86
        ).encode(
        alt.Y('open:Q'),
        alt.Y2('close:Q'),
        stroke=open_close_stroke
        )

    return rule + bar


def plot_dollar_candles(df, **kwargs):
    # inferred
    bin_size = df.V.iloc[0]
    x_title = f'${int(round(bin_size / 10**6))}M Bars'
    x_axis = alt.X(
        f'yearmonthdatehoursminutesseconds(T):O',
        axis=alt.Axis(title=x_title)
        )
    return plot_candles(df, x_axis=x_axis, **kwargs)


plot_dollar_candles(dfs['v'], width=1600, height=900)

alt.LayerChart(...)

In [7]:
def plot_deltas(df, y='D', width=800, height=400, **kwargs):
    return alt.Chart(df, width=width, height=height, **kwargs) \
              .mark_bar(fill='lightblue', stroke='grey', opacity=0.3, line=True) \
              .encode(
                  x=alt.X('yearmonthdatehoursminutesseconds(T):O', axis=alt.Axis(title='$10M Bars')),
                  y=y)

plot_dollar_candles(dfs['v'], width=1600, height=900) & \
plot_deltas(dfs['v'], width=1600, height=900)

alt.VConcatChart(...)